In [7]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import category_encoders as ce
from cleaning_functions_v2 import clean_complete
# from RMSPE import RMSPE
import time


import xgboost as xgb

In [8]:
def RMSPE(y_true, y_pred):
    """
    Calculate the root mean square percentage error

    INPUT:
    y_true: true values for the target variable
    y_pred: predicted values for the target variable

    OUTPUT:
    Value between 0 and 1, where 0 is the best and
    """
    y_difference = np.square((y_true - y_pred)/y_true)
    RMSPE = np.sqrt(np.mean(y_difference))
    return RMSPE

In [17]:
store_data = pd.read_csv("./data/store.csv")
train_data = pd.read_csv("./data/train.csv")
X_train,y_train,X_test,y_test = clean_complete(store_data,train_data)

X_train["Week"] = X_train["Week"].astype(int)
X_test["Week"] = X_test["Week"].astype(int)

/home/tjarke/anaconda3/envs/minichallenge/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/tjarke/anaconda3/envs/minichallenge/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/tjarke/anaconda3/envs/minichallenge/lib/python3.8/site-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/home/tjarke/anaconda3/envs/minichallen

In [18]:
parameters = {"learning_rate = ":[0.1,0.2,0.3,0.4],"n_estimators = ":[100,200,300,400]}

In [19]:
def xg_gridsearch(X_train,y_train,X_test,y_test,parameters):
    keys = []
    
    for i in parameters:
        keys.append(i)
    
    rmspe_best = 1
#     rmspe_train_best = 1
#     best_parameter_1 = 0
#     best_parameter_2 = 0
    
    
    for i in parameters[keys[0]]:
        for j in parameters[keys[1]]:
            xgbr = xgb.XGBRegressor(max_depth=5,learning_rate=i ,n_estimators=j,reg_lambda = 5,subsample = 0.8)           
            xgbr.fit(X_train,y_train)
            y_predict = xgbr.predict(X_test)
            rmspe_p = RMSPE(y_test,y_predict)
            y_train_error = xgbr.predict(X_train)
            rmspe_t = RMSPE(y_train,y_train_error)
            print(f"Model: {keys[0]} {i} and {keys[1]} {j} gives a train rmspe of {round(rmspe_t,3)} and a test rmspe of {round(rmspe_p,3)}")
            print("__________________________________________________________________")
            if rmspe_best > rmspe_p:
                rmspe_best = rmspe_p
                rmspe_train_best = rmspe_t
                best_parameter_1 = i
                best_parameter_2 = j
    print(f"The best combination is {keys[0]} {best_parameter_1} and {keys[1]} {best_parameter_2} and gives a train rmspe of {round(rmspe_train_best,3)} and a test rmspe of {round(rmspe_best,3)}")

In [20]:
xg_gridsearch(X_train,y_train,X_test,y_test,parameters)

Model: learning_rate =  0.1 and n_estimators =  100 gives a train rmspe of 0.349 and a test rmspe of 0.315
__________________________________________________________________
Model: learning_rate =  0.1 and n_estimators =  200 gives a train rmspe of 0.286 and a test rmspe of 0.265
__________________________________________________________________
Model: learning_rate =  0.1 and n_estimators =  300 gives a train rmspe of 0.247 and a test rmspe of 0.234
__________________________________________________________________
Model: learning_rate =  0.1 and n_estimators =  400 gives a train rmspe of 0.222 and a test rmspe of 0.215
__________________________________________________________________
Model: learning_rate =  0.2 and n_estimators =  100 gives a train rmspe of 0.284 and a test rmspe of 0.265
__________________________________________________________________
Model: learning_rate =  0.2 and n_estimators =  200 gives a train rmspe of 0.221 and a test rmspe of 0.219
_______________________